In [1]:
# To start mysql docker: sudo docker-compose up -d

import datajoint as dj
import os
import sys
import json


In [2]:
root_path = os.path.abspath(os.path.join(globals()['_dh'][0], '../..'))

In [3]:
local_cred_filename = os.path.join(root_path, 'datajoint/local_cred.ini')
with open(local_cred_filename) as f:
    local_cred = json.load(f)

In [4]:
local_cred

In [5]:
cred_filename = "~/"

dj.config['database.host'] = local_cred['host']
dj.config['database.user'] = local_cred['user']
dj.config['database.password'] = local_cred['password']

In [6]:
schema = dj.schema('test', locals())

In [7]:
schema = dj.schema('test', locals())
schema.drop()
schema = dj.schema('test', locals())

In [74]:
parent.drop()
child.drop()

In [9]:
# Testing multiple  

@schema
class Parent(dj.Manual):
    definition = """
    parent_id: int
    ---
    parent_data = NULL: varchar(20)
    """

@schema
class Middle(dj.Manual):
    definition = """
    -> Parent
    middle_id: int
    ---
    middle_data: varchar(20)
    """
    
@schema
class Child(dj.Manual):
    definition = """
    child_id: int
    ---
    child_data: varchar(20)
    """
    
    class ParentSrc(dj.Part):
        definition = """
        -> Middle
        -> Child
        """

## Testing many to one relationship

In [10]:
parent = Parent()
middle = Middle()
child = Child()

In [11]:
parent.insert([{'parent_id': 0, 'parent_data': 'stuff0'}])
parent.insert1({'parent_id': 1})
parent.insert([{'parent_id': 2}])
parent.insert1({'parent_id': 3, 'parent_data': 'stuff3'})
parent.insert1({'parent_id': 10, 'parent_data': 'stuff10'})
parent

In [12]:
p_id, p_data = parent.fetch('parent_id', 'parent_data')


In [13]:
middle.insert([{'parent_id': 0, 'middle_id': 0, 'middle_data': 'mid0'},
               {'parent_id': 1, 'middle_id': 1, 'middle_data': 'mid1'}])
middle

In [14]:
child.insert([{'child_id': 0, 'child_data': 'more0'},
              {'child_id': 1, 'child_data': 'more1'}])
child

In [15]:
child.ParentSrc.insert([{'parent_id': 0, 'middle_id': 0, 'child_id': 0},
                        {'parent_id': 1, 'middle_id': 1, 'child_id': 0},
                        {'parent_id': 0, 'middle_id': 0, 'child_id': 1},
                        {'parent_id': 1, 'middle_id': 1, 'child_id': 1}])
child.ParentSrc()

In [16]:
(child.ParentSrc() & 'child_id = 0') * child * parent

## Testing more realistic setup with many to one

In [114]:
@schema
class Env(dj.Manual):
    definition = """
    env_key: varchar(20)
    ---
    parent_data = NULL: varchar(20)
    env_key: varchar(20)
    """

@schema
class Epoch(dj.Manual):
    definition = """
    epoch_name: varchar(20)
    ---
    epoch_data = NULL: varchar(20)
    """

    
@schema
class Task1(dj.Manual):
    definition = """
    -> Env
    -> Epoch
    task1_id: int
    ---
    task1_data = NULL: int
    """

    
@schema
class Task2(dj.Manual):
    definition = """
    -> Env
    -> Epoch
    task2_id: int
    ---
    task2_data = NULL: int
    """    

@schema
class Tet(dj.Manual):
    definition = """
    -> Epoch
    tet_id: int
    ---
    tet_data = NULL: varchar(20)
    """
    
@schema
class LFP(dj.Manual):
    definition = """
    -> Tet
    ---
    lfp_data = NULL: varchar(20)
    """
    
@schema
class RipplesCons(dj.Manual):
    definition = """
    -> Epoch
    ---
    rip_cons_data = NULL: varchar(20)
    """
    class LFPSrc(dj.Part):
        definition = """
        -> RipplesCons
        -> LFP
        ---
        """

In [109]:
ep.drop(); env.drop()

In [85]:
ep = Epoch(); env = Env(); t1 = Task1(); t2 = Task2(); tet = Tet()

In [86]:
ep.insert1(['one', 'data1'])
ep.insert1(['two', 'data2'])
env.insert1(['env1','more1'])
env.insert1(['env2','more2'])
t1.insert1(['env1', 'one', 1, 1])
t2.insert1(['env2', 'two', 2, 2])
tet.insert1(['one', 1, 'tetdata1'])

In [87]:
display(ep)
display(env)
display(t1)
display(t2)
display(tet)

In [95]:
tet

In [122]:
dj.ERD(schema)

## Testing auto_increment

In [143]:
@schema
class Auto(dj.Lookup):
    definition = """
    id  :int auto_increment
    ---
    name :varchar(12)
    """
    contents = (
        dict(name="Godel"),
        dict(name="Escher"),
        dict(name="Bach")
    )

In [18]:
@schema
class Auto(dj.Manual):
    definition = """
    id: int auto_increment
    ---
    name :varchar(12)
    """

    def fill(self):
        if not self:
            self.insert([dict(name="Godel"), dict(name="Escher"), dict(name="Bach")])

Auto().insert([dict(name="Godel"), dict(name="Escher"), dict(name="Bach")])

In [149]:
Auto()

## Testing table classmethods

In [10]:
Test().drop()

In [52]:
@schema
class Above(dj.Manual):
    definition = """
    above_id: int
    """
    
@schema
class Test(dj.Imported):
    definition = """
    ->Above
    ---
    my_data: varchar(20)
    """
            
    def make(self, key):
        print(type(self).__name__)
        
    @classmethod
    def change_var(cls, new_var):
        print(type(cls))
        cls.class_var = new_var

    def test(self, new_var):
        print(self)
        print(type(self))
        print(new_var)


Test().change_var('mew')

In [55]:
Above.insert1([2])
Test().populate()

In [57]:
Test.__name__

In [26]:
Test()